In [2]:
import os
import json

In [4]:
file = '../../autofl/data/defects4j' 
script = ''
for file in sorted(os.listdir(file)):
    project, index = tuple(file.split('_'))
    index = int(index)
    script += f'cd /tmp/buggy_{project}_{index}\n'
    script += f'defects4j export -p "dir.src.classes" -o /src_dir/{project}_{index}.txt\n'
with open('d4j_src_dir.sh', 'w') as f:
    f.write(script)

In [ ]:
file = '../../autofl/data/defects4j' 
script = ''
for file in sorted(os.listdir(file)):
    project, index = tuple(file.split('_'))
    index = int(index)
    src_dir = "source" if project == 'Chart' else "src"
    if (project == 'Lang' and index >= 49 and index <= 65) or (project == 'Math' and index >= 85 and index <= 106):
        src_dir += '/java'
    elif project in ['Lang', 'Math', 'Time']:
        src_dir += '/main'
    script += f'python3 count_methods.py /tmp/buggy_{project}_{index}/{src_dir} -o methods/{project}_{index}.txt\n'
with open('d4j_methods.sh', 'w') as f:
    f.write(script)

In [6]:
dir = './src_dir' 
script = ''
for file in sorted(os.listdir(dir)):
    with open(os.path.join(dir, file)) as f:
        source_dir = f.read().strip()
    project, index = tuple(file.replace('.txt', '').split('_'))
    index = int(index)
    
    script += f'cloc /tmp/buggy_{project}_{index}/{source_dir} --csv > loc/{project}_{index}.csv\n'
with open('d4j_cloc.sh', 'w') as f:
    f.write(script)

# LOC

In [9]:
src_dir = 'loc'
projects = dict()
for file in sorted(os.listdir(src_dir)):
    project, index = tuple(file.replace('.csv', '').split('_'))
    if project not in projects:
        projects[project] = list()
    with open(os.path.join(src_dir, file)) as f:
        lines = f.readlines()
    line = [line for line in lines if ',Java,' in line]
    if len(line) == 1:
        projects[project].append(int(line[0].split(',')[-1]))

for key, value in projects.items():
    print(f'{key}\t{min(value)}--{max(value)}')

Chart	78564--96382
Closure	58989--104131
Lang	16593--21810
Math	9471--84317
Time	26589--27795


# Methods

In [28]:
src_dir = 'methods'
projects = dict()
for file in sorted(os.listdir(src_dir)):
    project, index = tuple(file.replace('.txt', '').split('_'))
    if project not in projects:
        projects[project] = list()
    with open(os.path.join(src_dir, file)) as f:
        line = f.readlines()[0]
    if not int(line.split(':')[-1].strip()):
        print(project, index)
    projects[project].append(int(line.split(':')[-1].strip()))

for key, value in projects.items():
    print(f'{key}\t{min(value)}--{max(value)}')

Chart	6378--8041
Closure	4621--8700
Lang	1794--2248
Math	1174--6015
Time	3535--3696


# Tests

In [29]:
src_dir = 'defects4j_original'
projects = dict()
for file in sorted(os.listdir(src_dir)):
    if not os.path.isdir(os.path.join(src_dir, file)):
        continue
    project, index = tuple(file.split('_'))
    if project not in projects:
        projects[project] = list()
    file = os.path.join(src_dir, file, 'test_snippet.json')
    with open(file) as f:
        lines = f.readlines()
    projects[project].append(len([0 for line in lines if '"signature"' in line and '.test' in line]))

for key, value in projects.items():
    print(f'{key}\t{min(value)}--{max(value)}')

Chart	1598--2201
Closure	2692--8625
Lang	1587--2265
Math	686--3548
Time	3802--4054
